In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import scipy.optimize as op
import scipy.signal as sig
from   scipy import stats
from   scipy import fftpack
import astropy
from   astropy.io import fits as pyfits
import random

import csv
import sys
import os
import importlib as imp
import glob
from   timeit import default_timer as timer
import warnings
import progressbar
import argparse
import json

import lightkurve as lk
import exoplanet as exo
import theano.tensor as T
import pymc3 as pm
import corner

from alderaan.constants import *
from alderaan.utils import *
from alderaan.Planet import *
from alderaan.LiteCurve import *
import alderaan.io as io
import alderaan.detrend as detrend
import alderaan.noise as noise


# flush buffer to avoid mixed outputs from progressbar
sys.stdout.flush()

# turn off FutureWarnings
warnings.filterwarnings('ignore', category=FutureWarning)

# start program timer
global_start_time = timer()

In [ ]:
# select mission, target, and paths

MISSION = "Kepler"
TARGET  = "K00137"
PRIMARY_DIR = '/Users/research/projects/alderaan/'
TRACE_FILE  = '/Users/research/projects/alderaan/__midway_traces//K00137/K00137_hbm_ttvs_200709a.fits'

# Make sure the necessary paths exist

In [ ]:
# directory in which to find lightcurve data
if MISSION == 'Kepler': DOWNLOAD_DIR = PRIMARY_DIR + 'MAST_downloads/'
if MISSION == 'Simulated': DOWNLOAD_DIR = PRIMARY_DIR + 'Simulations/'

# directories in which to place pipeline outputs    
FIGURE_DIR    = PRIMARY_DIR + 'Figures/' + TARGET + '/'
TRACE_DIR     = PRIMARY_DIR + 'Traces/' + TARGET + '/'
QUICK_TTV_DIR = PRIMARY_DIR + 'QuickTTVs/' + TARGET + '/'
DLC_DIR       = PRIMARY_DIR + 'Detrended_lightcurves/' + TARGET + '/'
NOISE_DIR     = PRIMARY_DIR + 'Noise_models/' + TARGET + '/'

# check if all the paths exist and create them if not
if os.path.exists(FIGURE_DIR) == False:
    os.mkdir(FIGURE_DIR)
    
if os.path.exists(TRACE_DIR) == False:
    os.mkdir(TRACE_DIR)
    
if os.path.exists(QUICK_TTV_DIR) == False:
    os.mkdir(QUICK_TTV_DIR)
    
if os.path.exists(DLC_DIR) == False:
    os.mkdir(DLC_DIR)
    
if os.path.exists(NOISE_DIR) == False:
    os.mkdir(NOISE_DIR)

# Get HBM TTV model posteriors

In [ ]:
with pyfits.open(TRACE_FILE) as hbm_trace:
    header = hbm_trace[0].header
    hdulist = pyfits.HDUList(hbm_trace)
    
    NPL = int((len(hdulist)-1)/3)
    NDRAWS = hbm_trace['POP_SD_0'].data.shape[0]
    
    print(NDRAWS, NPL)
    
    # TTV parameters
    OFFSET = [None]*NPL
    TTS    = [None]*NPL
    POP_SD = [None]*NPL
    
    for npl in range(NPL):
        try: OFFSET[npl] = hbm_trace['OFFSET_{0}'.format(npl)].data
        except: pass

        try: TTS[npl] = hbm_trace['TTS_{0}'.format(npl)].data
        except: pass
        
        try: POP_SD[npl] = hbm_trace['POP_SD_{0}'.format(npl)].data
        except: pass

In [ ]:
for npl in range(NPL):
    fig = corner.corner(POP_SD[npl]*24*60, labels=[0,1,2,3], color='C{0}'.format(npl))

# Get transit times, linear ephemeris and O-C

In [ ]:
for npl in range(NPL):
    tts = np.median(TTS[npl], 0)
    
    per = np.median(tts[1:]-tts[:-1])
    epoch = np.median(tts[0])

    inds = np.array(np.floor((tts - epoch + per/2)/per), dtype='int')
    
    
    pfit = np.polyfit(inds, tts, 1)
    ephem = np.polyval(pfit, inds)
    
    xtime = tts
    yomc  = (tts - ephem)*24*60
    yerr  = np.std(TTS[npl], 0)*24*60
    
    plt.figure(figsize=(15,4))
    plt.errorbar(xtime, yomc, yerr=yerr, fmt='.', color='C{0}'.format(npl))
    plt.xlim(100,1600)
    plt.ylim(ylim[npl])
    plt.xlabel("Time [days]")
    plt.ylabel("O-C")
    plt.show()
    